In [65]:
import numpy as np
import osmnx as ox
from osgeo import osr, ogr
import warnings

warnings.filterwarnings('ignore')

place_name = 'Пермь Индустриальный район'

In [66]:
geo_df = ox.footprints_from_place(place_name)

geo_df = geo_df[['building', 'building:levels', 'geometry']]
geo_df = geo_df.reset_index(drop=True)

geo_df = calculate_region_data(geo_df)

In [67]:
living_buildings_types = ['apartments', 'bungalow', 'cabin', 'detached', 'dormitory', 'farm', 'ger', 'hotel',
                           'house', 'houseboat', 'residential', 'semidetached_house', 'static_caravan', 'terrace']
real_live_buildings = geo_df.loc[geo_df.building.isin(living_buildings_types)]
living_building_dict = dict(real_live_buildings.building.value_counts())

In [68]:
population, fake_houses_count = calculate_people_count(geo_df, living_buildings_types)
print(f"В {place_name} проживает {round(population)} человек\n"
      f"Разновидности жилых зданий в {place_name}: {living_building_dict}, их количество {sum(living_building_dict.values())}\n"
      f"Предполагаемых жилых зданий в {place_name}: {fake_houses_count}")

Реальных людей посчитано 127594 человек
Предполагаемых людей посчитано 33033 человек
В Пермь Индустриальный район проживает 160628 человек
Разновидности жилых зданий в Пермь Индустриальный район: {'apartments': 641, 'house': 57, 'dormitory': 6, 'residential': 6, 'hotel': 2, 'detached': 1}, их количество 713
Предполагаемых жилых зданий в Пермь Индустриальный район: 482


In [25]:
def calculate_region_data(geo_df):
    geo_df["area"], geo_df["edges_count"] = None, None
    geo_df.loc[geo_df['building:levels'].isnull(), 'building:levels'] = 1
    for index, element in geo_df.iterrows():
        geo_df["area"][index] = get_area_from_polygon(element.geometry)
        geo_df["edges_count"][index] = get_edges_count(element.geometry)
    geo_df = geo_df.drop(['geometry'], axis=1)
    geo_df = geo_df.reset_index(drop=True)
    geo_df.edges_count = geo_df.edges_count.astype('float')
    geo_df = geo_df[np.isfinite(geo_df['edges_count'])]
    geo_df = geo_df.fillna(0)
    return geo_df

def get_edges_count(geometry_obj):
    if geometry_obj.type == 'MultiPolygon':
        count = 0
        for polygon in geometry_obj:
            count += polygon.exterior.coords.__len__()
    elif geometry_obj.type == 'Polygon':
        count = geometry_obj.exterior.coords.__len__()
    return count

In [7]:
def get_area_from_polygon(geometry_obj):
    if geometry_obj.type == 'MultiPolygon':
        area = 0
        for polygon in geometry_obj:
            area += __calculate_polygon_area(polygon)
    elif geometry_obj.type == 'Polygon':
        area = __calculate_polygon_area(geometry_obj)
    return area

def __calculate_polygon_area(polygon_obj):
    json = {
        'type': 'Polygon',
        'coordinates': [list(polygon_obj.exterior.coords)]
    }

    source = osr.SpatialReference()
    source.ImportFromEPSG(4326)
    target = osr.SpatialReference()
    target.ImportFromEPSG(5243)

    transform = osr.CoordinateTransformation(source, target)
    poly = ogr.CreateGeometryFromJson(str(json).replace('(', '[').replace(')', ']'))
    poly.Transform(transform)
    area = poly.GetArea()
    return area

In [55]:
def calculate_people_count(geo_df, living_buildings_types):
    fake_houses = 0
    real_people_count = 0
    fake_people_count = 0
    for index, row in geo_df.iterrows():
        if row['building'] in living_buildings_types:
            real_people_count += row.area * int(row['building:levels']) / 1.5 / 22
        elif int(row['building:levels']) > 1 and row['building'] == 'yes' and row['edges_count'] < 13:
            fake_people_count += row.area * int(row['building:levels']) / 3 / 22
            fake_houses += 1
    print('Реальных людей посчитано {} человек'.format(round(real_people_count)))
    print('Предполагаемых людей посчитано {} человек'.format(round(fake_people_count)))
    
    return real_people_count + fake_people_count , fake_houses